<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2025_sept/blob/main/4_pnl/clase_31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Visualizar textos en TensorFlow Projector: Paso a Paso en Google Colab

Este tutorial explica cómo transformar textos (por ejemplo, artículos constitucionales) en vectores para ser visualizados en [https://projector.tensorflow.org](https://projector.tensorflow.org).

---

## 🔹 Paso 1: Cargar tus textos

Simulamos una lista de artículos constitucionales:

```python
textos = [
    "Artículo 1: Chile es una república democrática, fundada en la soberanía popular.",
    "Artículo 2: La soberanía reside en el pueblo de Chile.",
    "Artículo 3: El Estado se organiza en tres poderes independientes.",
    "Artículo 4: La Constitución garantiza la igualdad ante la ley.",
    "Artículo 5: El medio ambiente debe ser protegido por el Estado.",
]
```

---

## 🔹 Paso 2: Instalar librerías necesarias

Usamos `sentence-transformers` para generar buenos embeddings multilingües:

```python
!pip install -q sentence-transformers
```

---

## 🔹 Paso 3: Convertir los textos a vectores (embeddings)

```python
from sentence_transformers import SentenceTransformer
import pandas as pd

# Cargar modelo multilingüe
modelo = SentenceTransformer('distiluse-base-multilingual-cased')

# Crear los vectores
vectores = modelo.encode(textos)

# Guardamos los textos como etiquetas para visualización
etiquetas = textos
```

---

## 🔹 Paso 4: Guardar archivos `.tsv`

```python
# Guardar los vectores en vectors.tsv
pd.DataFrame(vectores).to_csv("vectors.tsv", sep="\t", header=False, index=False)

# Guardar los textos originales como metadatos
pd.DataFrame(etiquetas).to_csv("metadata.tsv", sep="\t", header=False, index=False)
```

---

## 🔹 Paso 5: Descargar los archivos a tu computador

Puedes usar el explorador de archivos lateral en Colab o ejecutar:

```python
from google.colab import files
files.download("vectors.tsv")
files.download("metadata.tsv")
```

---

## 🔹 Paso 6: Cargar en TensorFlow Projector

1. Ir a: [https://projector.tensorflow.org/](https://projector.tensorflow.org/)
2. Hacer clic en “**Load**”
3. Subir:
   - `vectors.tsv` como **Tensor**
   - `metadata.tsv` como **Metadata**
4. Explorar con **PCA, t-SNE o UMAP**, y buscar similitudes entre textos.

---

## ✅ Opcional: Leer textos desde archivo `.txt` o `.csv`

Si tienes los artículos en un archivo `constitucion.txt` (una línea por artículo), haz:

```python
# Subir archivo
from google.colab import files
uploaded = files.upload()

# Leer textos desde el archivo
with open("constitucion.txt", encoding="utf-8") as f:
    textos = [line.strip() for line in f if line.strip()]
```

---

## 🎉 ¡Listo!
Ahora puedes explorar tus textos en un espacio semántico con TensorFlow Projector.


In [1]:
import os
import shutil

# Crear la carpeta 'corpus' si no existe
os.makedirs("corpus", exist_ok=True)

# Mover archivos PDF y EPUB cargados al entorno a la carpeta 'corpus'
for file in os.listdir():
    if file.endswith(".pdf") or file.endswith(".epub"):
        print(f"Moviendo {file} a carpeta corpus")
        shutil.move(file, os.path.join("corpus", file))

# Confirmar contenido de la carpeta
print("\nArchivos en la carpeta 'corpus':")
print(os.listdir("corpus"))



Archivos en la carpeta 'corpus':
[]


In [3]:
from google.colab import files

# Subir archivos .pdf y .epub desde tu equipo
uploaded = files.upload()

Saving gm_tomo2.epub to gm_tomo2.epub
Saving tomo3_e.epub to tomo3_e.epub
Saving tomo4_gm.epub to tomo4_gm.epub
Saving tomo5_gm(2).epub to tomo5_gm(2).epub
Saving tomo6_m-1.pdf to tomo6_m-1.pdf
Saving tomo7_m.pdf to tomo7_m.pdf
Saving tomo8_m.pdf to tomo8_m.pdf


In [4]:
%%capture
!pip install ebooklib PyMuPDF

In [5]:
import os
import shutil

# Crear carpeta corpus si no existe
os.makedirs("corpus", exist_ok=True)

# Mover todos los archivos .pdf y .epub que estén en la raíz a la carpeta corpus
for archivo in os.listdir():
    if archivo.endswith(".pdf") or archivo.endswith(".epub"):
        if not archivo.startswith("corpus"):  # Evitar mover de nuevo si ya están ahí
            print(f"Moviendo {archivo} a carpeta corpus/")
            shutil.move(archivo, os.path.join("corpus", archivo))

# Verificación
print("\nContenido de la carpeta corpus:")
print(os.listdir("corpus"))

Moviendo tomo6_m-1.pdf a carpeta corpus/
Moviendo tomo5_gm(2).epub a carpeta corpus/
Moviendo tomo4_gm.epub a carpeta corpus/
Moviendo tomo3_e.epub a carpeta corpus/
Moviendo tomo8_m.pdf a carpeta corpus/
Moviendo gm_tomo1.epub a carpeta corpus/
Moviendo gm_tomo2.epub a carpeta corpus/
Moviendo tomo7_m.pdf a carpeta corpus/

Contenido de la carpeta corpus:
['tomo6_m-1.pdf', 'tomo5_gm(2).epub', 'tomo4_gm.epub', 'tomo3_e.epub', 'tomo8_m.pdf', 'gm_tomo1.epub', 'gm_tomo2.epub', 'tomo7_m.pdf']


In [6]:
# 1. Librerías necesarias
import os
import fitz  # PyMuPDF
from ebooklib import epub, ITEM_DOCUMENT
from bs4 import BeautifulSoup

def extract_epub_text(path):
    book = epub.read_epub(path)
    text = ''
    for item in book.get_items():
        if item.get_type() == ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_content(), 'html.parser')
            text += soup.get_text()
    return text


# 3. Función para PDF
def extract_pdf_text(path):
    text = ''
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text

# 4. Extraer textos
corpus_path = "corpus"
textos_completos = ""

for archivo in os.listdir(corpus_path):
    ruta = os.path.join(corpus_path, archivo)
    if archivo.endswith(".epub"):
        print(f"Extrayendo EPUB: {archivo}")
        textos_completos += extract_epub_text(ruta) + "\n"
    elif archivo.endswith(".pdf"):
        print(f"Extrayendo PDF: {archivo}")
        textos_completos += extract_pdf_text(ruta) + "\n"

print("\nLongitud total del corpus:", len(textos_completos), "caracteres")

# 5. Guardar en archivo
with open("gabriela_mistral_corpus.txt", "w", encoding="utf-8") as f:
    f.write(textos_completos)

Extrayendo PDF: tomo6_m-1.pdf
Extrayendo EPUB: tomo5_gm(2).epub
Extrayendo EPUB: tomo4_gm.epub
Extrayendo EPUB: tomo3_e.epub
Extrayendo PDF: tomo8_m.pdf
Extrayendo EPUB: gm_tomo1.epub
Extrayendo EPUB: gm_tomo2.epub
Extrayendo PDF: tomo7_m.pdf

Longitud total del corpus: 4647275 caracteres


In [7]:
# Cargar corpus
with open("gabriela_mistral_corpus.txt", "r", encoding="utf-8") as f:
    corpus = f.read()

# Limpiar y tokenizar
import re
tokens = re.findall(r'\b\w+\b', corpus.lower())  # palabras minúsculas sin signos
print(f"Número de tokens: {len(tokens)}")

Número de tokens: 840658


In [8]:
!pip install -q sentence-transformers

In [9]:
# Supongamos que etiquetas es un solo string largo
with open("gabriela_mistral_corpus.txt", encoding="utf-8") as f:
    texto_largo = f.read()

# O si ya lo tienes cargado:
# texto_largo = etiquetas

# Dividir por líneas con contenido
etiquetas = [line.strip() for line in texto_largo.splitlines() if line.strip()]

In [10]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Cargar modelo multilingüe
modelo = SentenceTransformer('distiluse-base-multilingual-cased')

# Crear los vectores
vectores = modelo.encode(corpus)

# Guardamos los textos como etiquetas para visualización
etiquetas = corpus

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [12]:
print(len(etiquetas))
print(etiquetas[:5])

4647275
S E L


In [14]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Cargar el texto
with open("gabriela_mistral_corpus.txt", "r", encoding="utf-8") as f:
    frases = [line.strip() for line in f if line.strip()]

# Generar embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
vectors = model.encode(frases)

# Guardar archivos para TensorFlow Projector
pd.DataFrame(vectors).to_csv("vectors.tsv", sep="\t", header=False, index=False)
pd.DataFrame({'text': frases}).to_csv("metadata.tsv", sep="\t", header=False, index=False)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
from google.colab import files
files.download("vectors.tsv")
files.download("metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# ============================================================
# 1. Importar librerías necesarias
# ============================================================
import re
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# ============================================================
# 2. Cargar el corpus de Gabriela Mistral
# ============================================================
with open("gabriela_mistral_corpus.txt", "r", encoding="utf-8") as f:
    texto = f.read()

# ============================================================
# 3. Preprocesamiento del texto
#    - Eliminar números
#    - Eliminar caracteres no alfabéticos
#    - Convertir a minúsculas
# ============================================================
texto = re.sub(r"\d+", "", texto)                # eliminar números
texto = re.sub(r"[^A-Za-zÁÉÍÓÚáéíóúÑñüÜ\s]", "", texto)  # eliminar signos
texto = texto.lower()                            # pasar a minúsculas

# ============================================================
# 4. Tokenización por palabra (no por frase)
# ============================================================
# Dividir en palabras (puedes cambiarlo por split() o usar nltk.word_tokenize)
palabras = texto.split()
# Remover duplicados opcionalmente
palabras_unicas = list(set(palabras))

print(f"Total de palabras únicas: {len(palabras_unicas)}")

# ============================================================
# 5. Generar embeddings por palabra
# ============================================================
modelo = SentenceTransformer("all-MiniLM-L6-v2")
vectores = modelo.encode(palabras_unicas)

# ============================================================
# 6. Guardar archivos para TensorFlow Projector
# ============================================================
# Cada fila del TSV de vectores corresponde a una palabra
pd.DataFrame(vectores).to_csv("vectors.tsv", sep="\t", header=False, index=False)

# El archivo de metadata contiene las etiquetas (las palabras)
pd.DataFrame({'word': palabras_unicas}).to_csv("metadata.tsv", sep="\t", header=False, index=False)

print("Listo. Archivos 'vectors.tsv' y 'metadata.tsv' generados correctamente.")

files.download("vectors.tsv")
files.download("metadata.tsv")


Total de palabras únicas: 51322
Listo. Archivos 'vectors.tsv' y 'metadata.tsv' generados correctamente.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>